In [1]:
import cv2 as cv
import numpy as np
import pandas as pd

In [2]:
##cap = cv.VideoCapture(0)
whT = 320
confThreshold =0.5
nmsThreshold= 0.2
bbox=[]
csv_path = 'colors.csv'
index = ['color', 'color_name', 'hex', 'R', 'G', 'B']
df = pd.read_csv(csv_path, names=index, header=None)

In [3]:
#### LOAD MODEL
## Coco Names
classesFile = "coconames.txt"
classNames = []
with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
print(classNames)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [4]:
#####model files
modelConfiguration = "yolov3-320.cfg"
modelWeights = "yolov3-320.weights"
net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)
 

In [3]:
def get_color_name(B, G, R):
    minimum = 1000
    for i in range(len(df)):
        d = abs(R - int(df.loc[i,'R'])) + abs(G - int(df.loc[i,'G'])) + abs(B - int(df.loc[i,'B']))
        if d <= minimum:
            minimum = d
            cname = df.loc[i, 'color_name']

    return cname

In [9]:

def findObjects(outputs,img):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w,h = int(det[2]*wT) , int(det[3]*hT)
                x,y = int((det[0]*wT)-w/2) , int((det[1]*hT)-h/2)
                bbox.append([x,y,w,h])
                classIds.append(classId)
                confs.append(float(confidence))
 
    indices = cv.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)
 
    for i  in indices:
        i = i[0]
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
              # print(x,y,w,h)
        cv.rectangle(img, (x, y), (x+w,y+h), (255, 0 , 255), 2)
        imgcrop=img[int(hT-(y+h)+1 + 0.2*h):int(hT-y+1-0.2*h),int(x + 0.2*w):int(x+w-0.2*w)]
        imch,imcw,imcc=imgcrop.shape
        imgcpnp=np.array(imgcrop)
        bluearray=np.empty([imch,imcw])
        greenarray=np.empty([imch,imcw])
        redarray=np.empty([imch,imcw])
        
        for j in range(imch):
            for k in range(imcw):
                bluearray[j][k]=imgcpnp[j][k][0]
                greenarray[j][k]=imgcpnp[j][k][1]
                redarray[j][k]=imgcpnp[j][k][2]
                
        bluestd, bluemean=0.8*np.std(bluearray), np.mean(bluearray)
        greenstd, greenmean=0.8*np.std(greenarray),np.mean(greenarray)
        redstd, redmean =0.8*np.std(redarray), np.mean(redarray)
        bs=gs=rs=bn=gn=rn=0
        
        for l in range(imch):
            for m in range(imcw):
                if (bluearray[l][m] > bluemean-bluestd ) and (bluearray[l][m] < bluemean+bluestd):
                    bs=bs + bluearray[l][m]
                    bn=bn + 1
                    
                if (greenarray[l][m] > greenmean - greenstd) and ( greenarray[l][m]  < greenmean + greenstd):
                    gs=gs + greenarray[l][m]
                    gn=gn+1
                   
                if ( redarray[l][m] > redmean - redstd) and ( redarray[l][m] < redmean + redstd):
                    rs=rs + redarray[l][m]
                    rn=rn + 1
                   
               
           
        b=r=g=255
        if (bn!=0):
            b=bs/bn
        if (gn!=0):
            g=gs/gn
        if (rn!=0):
            r=rs/rn
        colorname=get_color_name(r,g,b)+ "      "
        
        
        
        
        cv.putText(img,f'{colorname.upper()}{classNames[classIds[i]].upper()} {int(confs[i]*100)}%',(x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
        
        
    


In [10]:
##while True:
   ## success, img = cap.read()
path= "redvelvet.jpg"
img= cv.imread(path)
blob = cv.dnn.blobFromImage(img, 1 / 255, (whT, whT), [0, 0, 0], 1, crop=False)
net.setInput(blob)
layersNames = net.getLayerNames()
outputNames = [(layersNames[i[0] - 1]) for i in net.getUnconnectedOutLayers()]
outputs = net.forward(outputNames)
findObjects(outputs,img)
 
cv.imshow('Image', img)
cv.waitKey(0)

-1

In [4]:
get_color_name(26.43, 197.94 , 233.16)

'Citrine'

In [5]:
get_color_name(37.76, 36.46, 30.66)

'Dark Jungle Green'

In [6]:
get_color_name(127.73, 153.50, 180.06)

'Pale Taupe'

In [7]:
get_color_name(232.36, 232.64, 232.88)

'Platinum'

In [8]:
get_color_name(173.52912127, 111.94848517 , 55.35795893)

'Steel Blue'

In [4]:
get_color_name(62.04435935 , 84.77766249 ,148.01678219)

'Smokey Topaz'